# Installation

In [3]:
# 적용 후 런타임 > 세션 다시 시작
!pip install -U \
     datasets==2.20.0 \
     evaluate==0.4.0 \
     scikit-learn==1.4.2

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
    

# 5주차 LLM 과제 설명

# 드라이브로 사본 복사해서 과제 풀고 github 에 ipynb 파일로 올려주세요

**문제는 총 2문제로 LLM 관련 1문제, RAG 1문제로 각각 소문제로 빈칸 채우기 실습 및 내용 서술 문제로 이루어져있습니다. 각각의 문제 설명에 알맞게 풀어주세요.**


# 1. LLM

## 빈칸을 채워서 모델을 로드해주세요

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    bos_token="</s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    mask_token="<mask>"
)
model = AutoModelForCausalLM.from_pretrained(model_name)
model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=51200, bias=False)
)

In [2]:
from datasets import load_dataset

split_dict = {
    "train": "train[:8000]",
    "test": "train[8000:10000]",
    "unused": "train[10000:]",
}
dataset = load_dataset("heegyu/kowikitext", split=split_dict) # custom code y 입력해주세요
del dataset["unused"]
dataset

The repository for heegyu/kowikitext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/heegyu/kowikitext.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['id', 'revid', 'url', 'title', 'text'],
        num_rows: 2000
    })
})

## 아래 train, test Dataset 에서 feautures 중 input_ids, attention_mask 각각의 역할을 서술해주세요

## 정답: input_ids는 실제 모델에 입력되는 토큰 ID의 seqence이다. 즉, 텍스트가 토큰화하여 숫자로 변환된 결과로, 모델은 이 숫자들을 기반으로 각 단어에 대한 의미 정보를 학습한다. attention_mask의 경우, 모델에게 어떤 토큰에 집중해야 하는 지 알려주는 역할을 한다. 의미있는 토큰에만 값을 부여하여 그 토큰에만 집중하도록 유도한다.

In [3]:
tokenized_dataset = dataset.map(
    lambda batch: tokenizer([f"{ti}\n{te}" for ti, te in zip(batch["title"], batch["text"])]),
    batched=True,
    num_proc=2,
    remove_columns=dataset["train"].column_names,
)
tokenized_dataset

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 8000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [4]:
max_length = 512
def group_texts(batched_sample):
    sample = {k: v[0] for k, v in batched_sample.items()}

    if sample["input_ids"][-1] != tokenizer.eos_token_id:
        for k in sample.keys():
            sample[k].append(
                tokenizer.eos_token_id if k == "input_ids" else sample[k][-1]
            )

    result = {k: [v[i: i + max_length] for i in range(0, len(v), max_length)] for k, v in sample.items()}
    return result

grouped_dataset = tokenized_dataset.map(
    group_texts,
    batched=True,
    batch_size=1,
    num_proc=2,
)
print(len(grouped_dataset["train"][0]["input_ids"]))
print(grouped_dataset)

Map (num_proc=2):   0%|          | 0/8000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/2000 [00:00<?, ? examples/s]

512
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 18365
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 2400
    })
})


In [5]:
from transformers import DataCollatorForLanguageModeling

collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
sample = collator([grouped_dataset["train"][i] for i in range(1)])

## 빈칸을 채워 결과를 출력해주세요

In [6]:
inputs = tokenizer("지난해 7월, ", return_tensors="pt").to(model.device)

outputs = model.generate(inputs.input_ids, max_new_tokens=100)
result = tokenizer.batch_decode(outputs, skip_special_tokens=True)
print(result[0])

지난해 7월, 롯데백화점 본점 지하 1층 식품매장에서 판매된 '롯데 햄버거' 제품에서 대장균이 검출돼 판매 중단된 바 있다.
롯데백화점 측은 "햄버거 판매 중단은 롯데백화점 본점 식품매장의 위생과 안전관리에 대한 고객들의 신뢰가 크게 훼손된 데 따른 것"이라며 "롯데백화점 본점 식품매장은 햄버거 판매 중단을 즉각 중단하고, 롯데백화점 본점 식품


In [7]:
import torch

input_ids = tokenizer("지난해 7월, ", return_tensors="pt").to(model.device).input_ids

with torch.no_grad():
    for _ in range(100):
        next_token = model(input_ids).logits[0, -1:].argmax(-1)
        input_ids = torch.cat((input_ids[0], next_token), -1).unsqueeze(0)

print(tokenizer.decode(input_ids[0].tolist()))

지난해 7월, 롯데백화점 본점 지하 1층 식품매장에서 판매된 '롯데 햄버거' 제품에서 대장균이 검출돼 판매 중단된 바 있다.
롯데백화점 측은 "햄버거 판매 중단은 롯데백화점 본점 식품매장의 위생과 안전관리에 대한 고객들의 신뢰가 크게 훼손된 데 따른 것"이라며 "롯데백화점 본점 식품매장은 햄버거 판매 중단을 즉각 중단하고, 롯데백화점 본점 식품


# 2. RAG

In [8]:
!pip install -q langchain langchain_community sentence-transformers faiss-cpu transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 23.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 772.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5

In [9]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 10.6 MB/s eta 0:00:00


In [10]:
import os
import tempfile
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain_community.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from google.colab import files
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline, BitsAndBytesConfig

In [11]:
# 파일 업로드 함수
def upload_file():
    print("문서 파일을 업로드해주세요.")
    uploaded = files.upload()
    file_path = list(uploaded.keys())[0]
    print(f"업로드 완료: {file_path}")
    return file_path

In [12]:
# LLM 모델 로컬 구성
def get_local_llm():
    print("언어 모델 로딩 중... (처음 실행 시 모델 다운로드에 시간이 걸릴 수 있습니다)")
    model_id = "google/flan-t5-base"  # 더 작은 모델로 변경

    quantization_config = BitsAndBytesConfig(
        load_in_8bit=True,
        llm_int8_enable_fp32_cpu_offload=True
    )
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_id, device_map="auto" # 수정된 부분
)

    pipe = pipeline(
        "text2text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,
        temperature=0.5
    )

    llm = HuggingFacePipeline(pipeline=pipe)
    print("언어 모델 로딩 완료!")
    return llm

## 아래 코드를 참고하여 RAG 프로세스를 서술해주세요

## 정답 : 원하는 파일 문서를 불러와 시스템이 참조할 원본 정보를 입력으로 넣는다. text_splitter를 통해 킨 문서를 모델이 처리할 수 있는 단위로 나누어 준다. embeddings를 통해 각 텍스트 청크를 벡터로 변환고 저장하여 질문과 유사한 문서 청크를 빠르게 찾을 수 있도록 한다. retriever를 사용하여 몇개의 유사 청크를 변환할 지 결정한다. LLM 모델을 설정해 로컬 파이프라인을 만들고 프롬프트 템플릿을 지정하여 질문에 맞는 답변을 생성하도록 유도한다. 이후 검색기와 LLM을 연결한 rag qa체인을 구축하여 문서 검색 -> 문맥 삽입 -> 답변 생성의 흐름을 자동화시킨다. 입력 쿼리와 생성과정을 통해 답변을 생성하고 생성한 정답은 벡터 인덱스로 로컬에 저장하여 재사용을 빠르게 할 수 있게 한다.

In [13]:
# RAG 시스템 구현
def colab_rag_system(file_path):
    # 1. 문서 로드
    loader = TextLoader(file_path)
    documents = loader.load()

    print(f"문서 로딩 완료: {len(documents)} 개의 문서")

    # 2. 문서 분할
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=100,
        length_function=len
    )
    texts = text_splitter.split_documents(documents)

    print(f"문서 분할 완료: {len(texts)} 개의 청크")

    # 3. 임베딩 생성 및 벡터 저장소 구축
    print("임베딩 모델 로딩 중...")
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

    print("벡터 저장소 구축 중...")
    vectorstore = FAISS.from_documents(texts, embeddings)

    # 4. 검색기 생성
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

    # 5. LLM 설정 - 로컬 파이프라인 사용
    llm = get_local_llm()

    # 6. 프롬프트 템플릿 정의
    prompt_template = """
    다음 정보를 바탕으로 질문에 답해주세요:

    {context}

    질문: {question}
    답변:
    """

    PROMPT = PromptTemplate(
        template=prompt_template,
        input_variables=["context", "question"]
    )

    # 7. RAG 파이프라인 구축
    print("RAG 파이프라인 구축 중...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": PROMPT}
    )

    print("RAG 시스템 준비 완료!")

    # 8. 대화형 인터페이스
    while True:
        query = input("\n질문을 입력하세요 (종료하려면 'q' 입력): ")

        if query.lower() == 'q':
            break

        result = qa_chain({"query": query})

        print("\n답변:", result["result"])
        print("\n관련 문서 출처:")
        for i, doc in enumerate(result["source_documents"]):
            print(f"문서 {i+1}: {doc.page_content[:100]}...")

    # 9. 벡터 저장소 저장
    with tempfile.TemporaryDirectory() as temp_dir:
        index_path = os.path.join(temp_dir, "faiss_index")
        vectorstore.save_local(index_path)
        print(f"\n인덱스를 '{index_path}'에 저장했습니다.")

        # 인덱스 다운로드 제공
        print("벡터 인덱스를 다운로드하려면 아래 코드를 실행하세요:")
        print("files.download(f'{index_path}/index.faiss')")
        print("files.download(f'{index_path}/index.pkl')")


## sample.md 를 업로드해서 아래 샘플 질문들을 입력해 결과를 출력하세요
1. "인공지능의 역사에서 튜링 테스트란 무엇인가요?"
2. "딥러닝 혁명은 언제 시작되었나요?"
3. "알파고란 무엇인가요?"
4."AI의 현대 응용 분야는 어떤 것들이 있나요?"
5."AI가 직면한 미래 과제는 무엇인가요?"

In [14]:
if __name__ == "__main__":
    file_path = upload_file()
    colab_rag_system(file_path)

문서 파일을 업로드해주세요.


Saving sample-document (1).md to sample-document (1).md
업로드 완료: sample-document (1).md
문서 로딩 완료: 1 개의 문서
문서 분할 완료: 2 개의 청크
임베딩 모델 로딩 중...


<ipython-input-13-2d8da98ea606>:21: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...


답변:                                                                                                                                                                                                                                                                

관련 문서 출처:
문서 1: ## 딥러닝 혁명 (2010년대-현재)
2010년대에 들어서면서 딥러닝이 AI 발전의 핵심이 되었습니다. 특히 2012년 ImageNet 대회에서 알렉스넷(AlexNet)의 성공은...
문서 2: # 인공지능의 역사와 발전

## 초기 인공지능 (1940-1950년대)
인공지능(AI)의 개념은 1940년대 후반부터 시작되었습니다. 앨런 튜링이 1950년에 발표한 논문 "Co...

질문을 입력하세요 (종료하려면 'q' 입력): AI의 현대 응용 분야는 어떤 것들이 있나요?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



답변:                                                                                                                                                                                                                                                                

관련 문서 출처:
문서 1: # 인공지능의 역사와 발전

## 초기 인공지능 (1940-1950년대)
인공지능(AI)의 개념은 1940년대 후반부터 시작되었습니다. 앨런 튜링이 1950년에 발표한 논문 "Co...
문서 2: ## 딥러닝 혁명 (2010년대-현재)
2010년대에 들어서면서 딥러닝이 AI 발전의 핵심이 되었습니다. 특히 2012년 ImageNet 대회에서 알렉스넷(AlexNet)의 성공은...

질문을 입력하세요 (종료하려면 'q' 입력): AI가 직면한 미래 과제는 무엇인가요?


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



답변:                                                                                                                                                                                                                                                                

관련 문서 출처:
문서 1: # 인공지능의 역사와 발전

## 초기 인공지능 (1940-1950년대)
인공지능(AI)의 개념은 1940년대 후반부터 시작되었습니다. 앨런 튜링이 1950년에 발표한 논문 "Co...
문서 2: ## 딥러닝 혁명 (2010년대-현재)
2010년대에 들어서면서 딥러닝이 AI 발전의 핵심이 되었습니다. 특히 2012년 ImageNet 대회에서 알렉스넷(AlexNet)의 성공은...

질문을 입력하세요 (종료하려면 'q' 입력): q

인덱스를 '/tmp/tmpc72qlfg1/faiss_index'에 저장했습니다.
벡터 인덱스를 다운로드하려면 아래 코드를 실행하세요:
files.download(f'{index_path}/index.faiss')
files.download(f'{index_path}/index.pkl')
